## Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.


## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

**Answer: `19`**

In [1]:
# import required libraries

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import Lasso, Ridge

from sklearn.metrics import mean_squared_error

import pickle

In [2]:
df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df.shape

(3066766, 19)

In [3]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

**Answer: `42.59`**

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [5]:
df.loc[:, 'tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df.loc[:, 'tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

df.duration = df.duration.apply(lambda td:td.total_seconds() / 60)

df.duration.describe().round(2)

count    3066766.00
mean          15.67
std           42.59
min          -29.20
25%            7.12
50%           11.52
75%           18.30
max        10029.18
Name: duration, dtype: float64

In [6]:
df.duration.std()

np.float64(42.59435124195458)

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

**Answer: `98%`**

In [7]:
# look at percentile
df.duration.describe(percentiles=[0.95, 0.98, 0.99]).round(2)

count    3066766.00
mean          15.67
std           42.59
min          -29.20
50%           11.52
95%           36.47
98%           48.73
99%           57.25
max        10029.18
Name: duration, dtype: float64

In [8]:
df_no_outliers = df[(df.duration >= 1) & (df.duration <= 60)]

In [9]:
# 3. Calculate the fraction
original_length = len(df)
new_length = len(df_no_outliers)
fraction_remaining = new_length / original_length

print(f"Fraction of records remaining: {fraction_remaining}")

Fraction of records remaining: 0.9812202822125979


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

**Answer: `515`**

In [10]:
categorical = ["PULocationID", "DOLocationID"]
# numerical = ["trip_distance"]

In [11]:
df_no_outliers = df_no_outliers.copy()

In [12]:
df_no_outliers[categorical] = df_no_outliers[categorical].astype(str)

In [13]:
# Create train dict
train_dicts = df_no_outliers[categorical].to_dict(orient='records')

In [14]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [15]:
X_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6018346 stored elements and shape (3009173, 515)>

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters, where duration is the response variable
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

**Answer: `7.64`**

In [16]:
target = "duration"
y_train = df_no_outliers[target].values

In [17]:
y_train

array([ 8.43333333,  6.31666667, 12.75      , ..., 24.51666667,
       13.        , 14.4       ], shape=(3009173,))

In [18]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [19]:
y_pred = lr.predict(X_train)

In [20]:
y_pred

array([11.52717387, 10.897809  , 11.32577005, ..., 11.73766216,
       12.70526846, 11.54252833], shape=(3009173,))

In [21]:
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
rmse

np.float64(7.649261932106969)

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

**Answer: `7.81`**

In [22]:
df_val = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

In [23]:
df_val.shape

(2913955, 19)

In [24]:
df_val.loc[:, 'tpep_dropoff_datetime'] = pd.to_datetime(df_val['tpep_dropoff_datetime'])
df_val.loc[:, 'tpep_pickup_datetime'] = pd.to_datetime(df_val['tpep_pickup_datetime'])

df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime

df_val.duration = df_val.duration.apply(lambda td:td.total_seconds() / 60)

In [25]:
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

In [26]:
df_val[categorical] = df_val[categorical].astype(str)

In [27]:
val_dicts = df_val[categorical].to_dict(orient='records')

In [28]:
X_val = dv.transform(val_dicts)

In [29]:
y_val = df_val[target].values

In [30]:
y_pred =lr.predict(X_val)

In [32]:
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
rmse

np.float64(7.811818743246608)